In [1]:
import xarray as xr
import numpy as np
import glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from xarray import DataArray
import pandas as pd
# from PyEMD import EEMD
import pylab as plt

In [12]:
na = np.load("/work/uo1075/u241321/data/eemd_NAO_DJF_dt.npy") # grid point detrend
spg = np.load("/work/uo1075/u241321/data/EEMD_spg_density_raw_c14.npy")  #
tend = np.load("/work/uo1075/u241321/data/eemd_tend_Norwegian.npy")
temp = np.load("/work/uo1075/u241321/data/eemd_t310_Norwegian.npy")
slheat = np.load("/work/uo1075/u241321/data/eemd_slheat_Norwegian.npy")
# transport = np.load("/work/uo1075/u241321/data/eemd_htransport310.npy") 
# transport1 = np.load("/work/uo1075/u241321/data/eemd_htransport310_ifc.npy") 
s_transport = np.load("/work/uo1075/u241321/data/stransport310_IFC_FSC.npy") 
slheat_djf = np.load("/work/uo1075/u241321/data/eemd_slheat_DJF_Norwegian.npy")
# nao = np.load("/work/uo1075/u241321/data/NAO_DJF_assi.npy")
# nao = np.load("/work/uo1075/u241321/data/eemd_NAO_DJF.npy")   #  


In [13]:
nao=na[2,:]+na[3,:]+na[4,:]

trans_fsc = np.load("/work/uo1075/u241321/data/eemd_htransport310.npy")
trans_ifc = np.load("/work/uo1075/u241321/data/eemd_htransport310_ifc.npy")

transport = trans_fsc[2,:] + trans_ifc[2,:]+ trans_fsc[3,:] + trans_ifc[3,:]+ trans_fsc[4,:] + trans_ifc[4,:] 

np.corrcoef(nao,transport)

array([[1.        , 0.40514461],
       [0.40514461, 1.        ]])

In [2]:
spg1 = np.load("/work/uo1075/u241321/data/EEMD_spg_density.npy")   # (5,50) mode(input, c1-c4), time
spg = spg1[2,:]+spg1[3,:]+spg1[4,:]

In [12]:
na = np.load("/work/uo1075/u241321/data/eemd_NAO_DJF.npy")

In [4]:
np.corrcoef(na[0,:], trans_fsc[0,:]+ trans_ifc[0,:])

array([[1.        , 0.45225539],
       [0.45225539, 1.        ]])

In [7]:
np.corrcoef(na[0,:], s_transport)

array([[1.        , 0.53940842],
       [0.53940842, 1.        ]])

In [20]:
np.corrcoef(transport[0,:]+ transport1[0,:], temp[2,:])

array([[1.        , 0.38210369],
       [0.38210369, 1.        ]])

In [23]:
np.corrcoef(transport[0,:]+ transport1[0,:], temp[1,:]+temp[2,:]+temp[3,:])

array([[1.        , 0.34798698],
       [0.34798698, 1.        ]])

In [24]:
np.corrcoef(transport[0,:], temp[2,:])

array([[1.        , 0.24305276],
       [0.24305276, 1.        ]])

In [25]:
np.corrcoef(transport[1,:], temp[2,:])

array([[1.        , 0.13115228],
       [0.13115228, 1.        ]])

In [27]:
slheat.shape

(5, 49)

In [3]:
# Original script; with trend # result is almost same as the result of fft function
def rho(datax):
    # Calculates the lag-1 Autocorrelation Coefficient.
    import numpy as np  
    nrho=len(datax)
    sommesup=0
    sommeinf=0
    moy=np.sum(datax)/nrho
    datam=datax-moy
    for i in np.arange(1,nrho):
        j=i-1
        sommesup=sommesup+(datam[i]*datam[j])
        sommeinf=sommeinf+(datam[j]**2)
    rho=sommesup/sommeinf
    return rho

def rhoAlt(datax,dt=1):
    # Calculates the lag-dt Autocorrelation Coefficient, given the dt.
    import numpy as np  
    r=np.corrcoef(datax[0:-dt-1],datax[dt:-1])
    return r[0,1]

def rednoise(lenx, rho, nsim=1000, dist='normal', returnWhite=False, 
             mean=0, std=1, lo=-1, hi=1):
    '''
    Creates nsim time series of rednoise of length=lenx, with lag-1 autocorrelation rho.
    For normally-distributed series, user can provide mean and std.
    For uniformely-distributed series, user can provide low and high bounds.
    '''
    import numpy as np
    srho=(1-(rho**2))**(0.5)
    red=np.zeros((lenx,nsim))
    white=np.zeros((lenx,nsim))
    for j in range(nsim):
        for i in range(lenx):
            if dist=='normal':
                white[i,j]=np.random.normal(loc=mean, scale=std) 
            elif dist=='uniform':
                white[i,j]=np.random.uniform(low=lo, high=hi)
    for j in range(nsim):
        for i in range(lenx):
            if i==0:
                red[i,j]=white[i,j]*srho
            else:
                red[i,j]=rho*red[i-1,j]+white[i,j]*srho
    if returnWhite:
        return red, white
    else:
        return red
    
def periods(x,dt,returnPeriods=True, nsim=1000):
    import numpy as np
    from scipy import signal
    from scipy.stats.distributions import chi2
    
    # Calculate periodogram of x
    f, psd = signal.periodogram(x,fs=dt,detrend=False,scaling='spectrum')
    per=1/f
    max5=np.zeros((5,3))
    psdc=psd.copy()

    # Get the 5 larges periods
    for i in range(5):
        max5[i,0]=f[psdc==max(psdc)]
        max5[i,1]=psdc[psdc==max(psdc)]
        max5[i,2]=per[psdc==max(psdc)]
        psdc[psdc==max(psdc)]=0

    # Calculate periodograms of the nsim red-noise series
    fn=np.zeros((len(f),nsim))
    Pn=np.zeros((len(f),nsim))
    red=rednoise(len(x),rhoAlt(x,dt=dt),dist='normal',nsim=nsim)
    for i in range(nsim):
    #         if np.remainder(i,100)==0:
    #             print('%.0f %%' %(i/10))
        fn[:,i], Pn[:,i] = signal.periodogram(red[:,i],fs=dt,detrend=False,scaling='spectrum')

    # Mean spectrum of nsim simulations
    meanRed=np.mean(Pn,axis=1)

    # Get Percentiles of Distribution of Red-Noise Spectra
    pctl=np.zeros((len(f),4)) # 0.8, 0.9, 0.95, 0.99
    for i in range(len(f)):
        pctl[i,0]=np.percentile(Pn[i,:],80)
        pctl[i,1]=np.percentile(Pn[i,:],90)
        pctl[i,2]=np.percentile(Pn[i,:],95)
        pctl[i,3]=np.percentile(Pn[i,:],99)
     
   
    frequency = f
    spectrum = psd
    return frequency, spectrum, pctl, max5, meanRed



In [14]:
fre, spe, pct1, max5, meanred = periods(na[3,:],1,returnPeriods=True, nsim=1000)
max5[0,2]  # largest period

/tmp/ipykernel_799235/3280903895.py:58: RuntimeWarning: divide by zero encountered in divide
  per=1/f


10.0

In [8]:
t.shape[0]

50